# Time to do some data science!!!

Note that if you have pre-populated your database using the files in `json_files`, you can just start up with this notebook.

In [ ]:
%matplotlib inline

import json
import re
import urllib
from pprint import pprint
import time
from tqdm import tqdm

from py2neo import Node, Graph, Relationship, NodeMatcher
from py2neo.bulk import merge_nodes

import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import plot_confusion_matrix

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
# If you are using a Sandbox instance, you will want to use the following (commented) line.  
# If you are using a Docker container for your DB, use the uncommented line.
# graph = Graph("bolt://some_ip_address:7687", name="neo4j", password="some_password")

graph = Graph("bolt://neo4j:7687", name="neo4j", password="kgDemo")
nodes_matcher = NodeMatcher(graph)

# Let's try some more entity disambiguation

Recall in the previous notebook that we looked at the cosine similarity of word vectors.  Instead, now let's look at the overlap of the relationships between our starting node, 'oh bah mə', and  'barack hussein obama ii'.  We might expect if there was strong similarity that we would see a lot of relationship overlap.  

In [ ]:
pbo_ls = []
pbo = graph.run('MATCH (n:Node {name: "president barack obama"})--(m) RETURN DISTINCT m.name')
for record in pbo:
    pbo_ls.append(record[0])
print('Total number of connected nodes: ', len(pbo_ls))
pbo_ls

In [ ]:
obm_ls = []
obm = graph.run('MATCH (n:Node {name: "oh bah mə"})--(m) RETURN DISTINCT m.name')
for record in obm:
    obm_ls.append(record[0])
print('Total number of connected nodes: ', len(obm_ls))

In [ ]:
pbo_set = set(pbo_ls)
obm_set = set(obm_ls)
if (pbo_set & obm_set):
    print('Number of overlapping elements: ', len(pbo_set & obm_set))
    print('Percent of overlapping elements: ', len(pbo_set & obm_set)/len(pbo_ls))
    print(pbo_set & obm_set)

# Observation

So we can see that 100% of the nodes connected to 'barack hussein obama ii' are in the connected node list of 'oh bah mə'.  This is a strong indicator that the former might be the same entity as the later.

# Now let's connect to the graph and do some ML

Here we are going to take advantage of the ability to both run Cypher queries in `py2neo` as well as to write the results to a Pandas DataFrame.

In [ ]:
df = graph.run('MATCH (n:Node) RETURN n.name, n.node_labels, n.pptu_person, n.pptu_place, n.pptu_thing, n.pptu_unknown, n.word_vec, n.n2v_all_nodes').to_data_frame()
df.columns = ['name', 'node_labels', 'pptu_person', 'pptu_place', 
              'pptu_thing', 'pptu_unknown', 'word_vec', 'n2v_all_nodes']
df2 = df.fillna(0)
df2.head()

# Create a variety of `X` variables for the model (in the style of `scikit-learn`)...

In [ ]:
def create_X(df2):

    word_vec_ls = df2['word_vec'].to_list()
    word_vec_arr = np.array([np.array(x) if x != 0 else np.zeros(300).tolist() for x in word_vec_ls], dtype=object)

    n2v_an_ls = df2['n2v_all_nodes'].to_list()
    n2v_arr = np.array([np.array(x) for x in n2v_an_ls], dtype=object)

    print(word_vec_arr.shape, n2v_arr.shape)
    
    return word_vec_arr, n2v_arr

In [ ]:
X_word_vec, X_all_nodes = create_X(df2)

# Out of convenience...

...we are creating this function that will run a support vector machine classifier to see how well the different embeddings do at predicting different labels.  We do note that this is a multi-label problem (Person, Place, Thing, Unknown), but for simplicity we are going to evaluate the prediction against single labels within that dataset.  The interested reader is encouraged to try more sophisticated models that can handle the multi-label problem better.

We also show below that each of the classes is significantly imbalanced.  The interested participant is encourage to experiment with balancing the classes to see how this impacts the overall accuracy.

In [ ]:
def modeler(df, column_name, X, k_folds=5, model='linear', show_matrix=True):
    
    y = df[column_name].fillna(0.0).to_numpy()
    acc_scores = []
    
    pos = np.count_nonzero(y == 1.0)
    neg = y.shape[0] - pos
    print('Number of positive: ', pos, ' Number of negative: ', neg)
    
    for i in range(0, k_folds):
        
        X_train, X_test, y_train, y_test = train_test_split(X_word_vec, y, test_size=0.25)
        clf = svm.SVC(kernel='linear')
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        acc = accuracy_score(pred, y_test)
        acc_scores.append(acc)        
        
    print('Accuracy scores: ', acc_scores)
    print('Mean accuracy: ', np.mean(acc_scores))
    
    if show_matrix:
        matrix = plot_confusion_matrix(clf, X_test, y_test, cmap=plt.cm.Blues, normalize='true')
        plt.show(matrix)
        plt.show()
        
    return

In [ ]:
modeler(df2, 'pptu_person', X_word_vec)

In [ ]:
modeler(df2, 'pptu_person', X_all_nodes)

In [ ]:
modeler(df2, 'pptu_place', X_word_vec)

In [ ]:
modeler(df2, 'pptu_place', X_all_nodes)

In [ ]:
modeler(df2, 'pptu_thing', X_word_vec)

In [ ]:
modeler(df, 'pptu_thing', X_all_nodes)

# Next steps

There are so many things that you can try from here!  Some of the things I might consider would be:

- Take time to tune the hyperparameters.  This can be done for:
  - The spacy word embeddings
  - The graph embeddings
  - The ML model
- Trying more sophisticated embedding approaches, such as GraphSAGE that takes into account the node properties.
- Explore different embeddings.  Here we used the spacy word vectors to create embeddings for the nodes, but there are many, many more ways to create vectors that could be used for training the ML models!  Get creative!
- Work the class imbalance problem.
- This graph is quite small in reality.  Work on growing the graph by adding more layers to it via either Wikipedia or the Google Knowledge Graph.  As the graph gets larger, we might expect that the graph embeddings approaches will start to really shine beyond the word embeddings.

